# Clustering and segmenting Neighborhood in Rennes, France

## Introduction / Business Problem

Rennes is my birth city and was quite in advance on its time by giving open datas to the people since at least ten years. Now, there is lots of data from the city of Rennes and other contractors that have business in Rennes. Like for example the company that run the bus in Rennes has some API to get datas about them. Bus stop location, real time traffic, etc..

Our objectif in this project is to cluster Rennes Neighborhood using foursquare and using datas from the city of Rennes.

We use foursquare API but we also add data about transportation of Rennes city.

Then we try to analyse cluster predicted and assigned them to groups of people. 

Overall business objectif  is to direct people/businesses, that are looking to settle in Rennes, into the good neighborhood.

## Data

In order to complete out goals, two source of datas are going to be used : 
- Foursquare API
- Rennes' datasets

All datas are quite easily available. Biggest challenge is to find out the neighborhood and assign each venue, structure, etc.. to the correct neighborhood.


### Foursquare API

Foursquare is a social location service that allows users to explore the world around them. They are at this time able to review locations in which they come and go and give a notation and a comment about this place. 

The Foursquare API allows application developers to interact with the Foursquare platform. We can retrieve venues and all the details about it (notations, comments, users, etc..), but also details about users.

In order to cluster our neighborhood, we will use Foursquare API to have data about Rennes locations and venues.
We will then be able to link top 10 venues to each neighborhood.

### Rennes' datasets

In addition to the Foursquare venues, we will add transport informations to our neighborhood : 
- Number of Bus stops
- Number of charger for electric vehicles
- Number of kilometers of paid parking
- Number of bike supports
- Number of kilometers of bike ways
- Number of culture equipment

Each of these datasets are coming from Rennes Open Data Services. We downloaded some csv files containing informations about Rennes and others cities surrounding Rennes. Each equipment, location, etc. is coming with a Latitude and a Longitude. for simplicity purpose, we will use in this study only the informations concerning Rennes. 

In Rennes Open Datas Services we were also able to find on very important dataset, which is the one that cut Rennes into neighborhood.

Thanksfully Rennes is a quite small city compared to New York and Toronto, so we do not need to cut down the numbers of neighborhood.

In order to analyse our clusters, we also have datas about : 
- Sexe, age and nationality of population in Rennes
- Seniority of habitants of Rennes

We will use these datas to see if our analysis is correct at the end of the project. 
We will indeed be able to see if habitants of a neighbourhood are concording with the analysis we made about each clusters we created.

# Notebook

## Data import

First we import all the libraries needed

In [1]:
import pandas as pd
import numpy as np
import json # library to handle JSON files
from math import *

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


After this we define this little function that will help us compute distance between two points defined with Longitude and Latitude. This will be very helpfull for us in order to define the appartenance of an object (bus stop, electric car charger, etc.) to a neighbourhood.

In [2]:
def haversine(lon1, lat1, lon2, lat2):

    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

## Neighbourhood

First we import the Neighbourhood of Rennes. We downloaded csv files from Rennes Open Data website into the Watson Studio. We use the code provided by IBM to import the data into the notebook. After This we only remove the unused column and we keep only the Latitude and Longitude (as floats), and of course the name of the neighbourhood.

In [3]:
# The code was removed by Watson Studio for sharing.

,nom,lat,long
0,Le Blosne,48.085013,-1.658945
1,Cleunay - Arsenal - Redon,48.095816,-1.722033
2,Saint Martin,48.126865,-1.683262
3,Villejean - Beauregard,48.129004,-1.711953
4,Bréquigny,48.086038,-1.685403


Let's see this on the map : 

In [4]:
latitude = 48.117266
longitude = -1.6777926

# create map of NToronto using latitude and longitude values
map_rennes = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(rennes_neighbourhood['lat'], rennes_neighbourhood['long'], rennes_neighbourhood['nom']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_rennes)  
    
map_rennes

## Other Datas from Rennes Open Data Website :

In all the datas that we will get in this section, we will remove unused columns and keep a column with the equipment name, the longitude and the latitude as float and we will add a column full of Nan named Neighbourhood in order to class each object into a neighbourhood later. And then we will show the head.

Also for simplicity we only keep datas from the city of Rennes only, as in some data frames, their is data of suburb of Rennes that we remove.

### Electric Car Charger

In [5]:
body = client_2551b64066e74033992250268342bdad.get_object(Bucket='courseraproject-donotdelete-pr-pstz2scfjmnlsh',Key='bornes-de-recharge-dediees-aux-vehicules-electriques-sur-le-territoire-de-rennes.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

electric_car_charger = pd.read_csv(body,delimiter=';')
electric_car_charger['addr'], electric_car_charger['town'] = electric_car_charger['site_adr'].str.split(',').str
electric_car_charger = electric_car_charger.loc[electric_car_charger['town'] == ' Rennes']
electric_car_charger['lat'], electric_car_charger['long'] = electric_car_charger['Geo Point'].str.split(',').str
electric_car_charger = electric_car_charger[['lat','long']]
electric_car_charger['Equipment'] = 'electric car charger'
electric_car_charger.head()

,lat,long,Equipment
2,48.1061492349,-1.67716224362,electric car charger
3,48.1109995579,-1.6836300089,electric car charger
4,48.1305281329,-1.6383229456,electric car charger
6,48.0924137533,-1.674211241,electric car charger
7,48.1135228591,-1.68623278686,electric car charger


### Bus stops

In [6]:
body = client_2551b64066e74033992250268342bdad.get_object(Bucket='courseraproject-donotdelete-pr-pstz2scfjmnlsh',Key='equipement-accessibilite-arrets-bus.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

bus_stop = pd.read_csv(body,delimiter=';')
bus_stop = bus_stop.loc[bus_stop['Commune (nom)'] == 'Rennes']
bus_stop['lat'], bus_stop['long'] = bus_stop['Coordonnées'].str.split(',').str
bus_stop = bus_stop[['lat','long']]
bus_stop['Equipment'] = 'bus stop'
bus_stop.head()

,lat,long,Equipment
0,48.127369,-1.640433,bus stop
1,48.121446,-1.655036,bus stop
2,48.119241,-1.667693,bus stop
3,48.11605,-1.674245,bus stop
4,48.11252,-1.680352,bus stop


### Bike Stops

For this one we keep the number of support at each point in order to sum them up later.

In [7]:
body = client_2551b64066e74033992250268342bdad.get_object(Bucket='courseraproject-donotdelete-pr-pstz2scfjmnlsh',Key='supports-velos.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

bike_stops = pd.read_csv(body,delimiter=';')
bike_stops = bike_stops.loc[bike_stops['nom_commune'] == 'Rennes']
bike_stops['lat'], bike_stops['long'] = bike_stops['Geo Point'].str.split(',').str
bike_stops = bike_stops[['nombre_support', 'lat', 'long']]
bike_stops['Equipment'] = 'bike stops'
bike_stops.head()

,nombre_support,lat,long,Equipment
0,4,48.1174872812,-1.6777579592,bike stops
1,5,48.1098139465,-1.67522515707,bike stops
2,8,48.1096012826,-1.67985481523,bike stops
3,5,48.113188555,-1.67762495861,bike stops
4,4,48.0859188837,-1.64220623748,bike stops


### Cultural equipment

For this one, name of the neighbourhood is already included. We do not even need to keep Latitude and Longitude.

In [8]:
body = client_2551b64066e74033992250268342bdad.get_object(Bucket='courseraproject-donotdelete-pr-pstz2scfjmnlsh',Key='liste-des-equipements-et-organismes-culturels-de-rennes-metropole.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

cultural_equipment = pd.read_csv(body,delimiter=';')
cultural_equipment = cultural_equipment.loc[cultural_equipment['CommuneNom'] == 'Rennes']
cultural_equipment['Equipment'] = 'Cultural equipement'
cultural_equipment['Neighbourhood'] = cultural_equipment['QuarNom']
cultural_equipment = cultural_equipment[['Equipment','Neighbourhood']]

cultural_equipment.head()

,Equipment,Neighbourhood
0,Cultural equipement,Centre
1,Cultural equipement,Thabor - Saint-Hélier - Alphonse Guérin
2,Cultural equipement,Maurepas - Bellangerais
3,Cultural equipement,Maurepas - Bellangerais
4,Cultural equipement,Jeanne d'Arc - Longs Champs - Beaulieu


### Green Roads

In [9]:
body = client_2551b64066e74033992250268342bdad.get_object(Bucket='courseraproject-donotdelete-pr-pstz2scfjmnlsh',Key='amenagements-velo-et-zones-de-circulation-apaisee-sur-rennes-metropole.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

green_roads = pd.read_csv(body,delimiter=';')

green_roads = green_roads.loc[green_roads['c_insee'] == 35238.0]
green_roads['lat'], green_roads['long'] = green_roads['Geo Point'].str.split(',').str
green_roads = green_roads[['lat','long']]
green_roads['Equipment'] = 'Green Roads'
green_roads.head()

,lat,long,Equipment
0,48.1260914882,-1.63356338546,Green Roads
1,48.1138992815,-1.67989604862,Green Roads
2,48.1125929026,-1.68146027273,Green Roads
3,48.1011841606,-1.67756709221,Green Roads
4,48.0909412214,-1.66796278621,Green Roads


### Paid Parking

In [10]:
body = client_2551b64066e74033992250268342bdad.get_object(Bucket='courseraproject-donotdelete-pr-pstz2scfjmnlsh',Key='portions-de-voies-en-stationnement-payant-sur-la-ville-de-rennes.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

paid_parking = pd.read_csv(body,delimiter=';')
paid_parking['lat'], paid_parking['long'] = paid_parking['Geo Point'].str.split(',').str
paid_parking = paid_parking[['lat','long']]
paid_parking['Equipment'] = 'Parking'
paid_parking.head()

,lat,long,Equipment
0,48.1123867128,-1.66129260255,Parking
1,48.1047938066,-1.69226198181,Parking
2,48.109626229,-1.67280206572,Parking
3,48.1041845846,-1.69095615868,Parking
4,48.1060532383,-1.69053215943,Parking


Lets merge all of our Data Frames (except cultural equipment that we will add once we found the neigbourhood of each equipment).
We also make sure that latitude and longitude are floats.

In [74]:
frames = [paid_parking, green_roads, bike_stops, bus_stop, electric_car_charger]
equipments = pd.concat(frames)
equipments['lat'] = equipments['lat'].astype(float)
equipments['long'] = equipments['long'].astype(float)
equipments = equipments.reset_index(drop=True)
equipments.head()

,Equipment,lat,long,nombre_support
0,Parking,48.112387,-1.661293,NaN
1,Parking,48.104794,-1.692262,NaN
2,Parking,48.109626,-1.672802,NaN
3,Parking,48.104185,-1.690956,NaN
4,Parking,48.106053,-1.690532,NaN


## Put equipment into Neighboorhood

For simplicity reason we will assign the equipment to the closest neighbourhood.

We will create column with name of the Neighbourhood.
For the computation of the distances we use the fonction defined at the begining.
We will compute distance for each equipment from each neighbourhood.

In [75]:
df = pd.get_dummies(rennes_neighbourhood,prefix='',prefix_sep='')
df.drop(df.index, inplace=True)
df = df.drop(['lat', 'long'], axis=1)
df.applymap(str)
equipments = pd.concat([equipments, df], axis=1)
equipments.head()

,Equipment,lat,long,nombre_support,Bourg l'Evesque - La Touche - Moulin du Comte,Bréquigny,Centre,Cleunay - Arsenal - Redon,Francisco Ferrer - Landry - Poterie,Jeanne d'Arc - Longs Champs - Beaulieu,Le Blosne,Maurepas - Bellangerais,Saint Martin,Sud gare,Thabor - Saint-Hélier - Alphonse Guérin,Villejean - Beauregard
0,Parking,48.112387,-1.661293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Parking,48.104794,-1.692262,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Parking,48.109626,-1.672802,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Parking,48.104185,-1.690956,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Parking,48.106053,-1.690532,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
# Version that should be faster
check_distance = equipments.copy()

for i in rennes_neighbourhood.index:
    check_distance['Neighbourhood']=rennes_neighbourhood.at(i,'nom')
    check_distance['Neighbourhood_lat']=rennes_neighbourhood.at(i,'lat')
    check_distance['Neighbourhood_long']=rennes_neighbourhood.at(i,'long')
    check_distance['dist_from_Neighbourhood']= check_distance.apply(lambda row: haversine(row['Neighbourhood_long'], 
                                            row['Neighbourhood_lat'], 
                                            row['long'], 
                                            row['lat']), axis=1)
    
    print(rennes_neighbourhood.at(i,'nom'))
    for j in check_distance.index: 
        equipments.loc[j,check_distance.loc[j,'Neighbourhood']] = check_distance.loc[j,'dist_from_Neighbourhood']
    
    print(rennes_neighbourhood.at(i,'nom'))
equipments.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:5: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:13: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Le Blosne


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:17: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Le Blosne
Cleunay - Arsenal - Redon
Cleunay - Arsenal - Redon
Saint Martin
Saint Martin
Villejean - Beauregard
Villejean - Beauregard
Bréquigny
Bréquigny
Jeanne d'Arc - Longs Champs - Beaulieu
Jeanne d'Arc - Longs Champs - Beaulieu
Sud gare
Sud gare
Bourg l'Evesque - La Touche - Moulin du Comte
Bourg l'Evesque - La Touche - Moulin du Comte
Maurepas - Bellangerais
Maurepas - Bellangerais
Thabor - Saint-Hélier - Alphonse Guérin
Thabor - Saint-Hélier - Alphonse Guérin
Francisco Ferrer - Landry - Poterie
Francisco Ferrer - Landry - Poterie
Centre
Centre


,Equipment,lat,long,nombre_support,Bourg l'Evesque - La Touche - Moulin du Comte,Bréquigny,Centre,Cleunay - Arsenal - Redon,Francisco Ferrer - Landry - Poterie,Jeanne d'Arc - Longs Champs - Beaulieu,Le Blosne,Maurepas - Bellangerais,Saint Martin,Sud gare,Thabor - Saint-Hélier - Alphonse Guérin,Villejean - Beauregard
0,Parking,48.112387,-1.661293,NaN,3.896795,3.433599,1.418882,4.872061,2.216049,1.572841,3.048843,2.762201,2.291612,1.948695,0.043649,4.189906
1,Parking,48.104794,-1.692262,NaN,1.690138,2.146864,1.131022,2.425713,3.758173,3.982923,3.310660,4.603510,2.543529,1.622550,2.450537,3.063282
2,Parking,48.109626,-1.672802,NaN,3.033481,2.784813,0.580635,3.965113,2.636811,2.455970,2.923974,3.350906,2.068178,1.407921,0.909966,3.617837
3,Parking,48.104185,-1.690956,NaN,1.804669,2.059528,1.102737,2.488301,3.647867,3.927924,3.193212,4.594616,2.585828,1.504398,2.383094,3.169517
4,Parking,48.106053,-1.690532,NaN,1.771707,2.257959,0.943917,2.601469,3.671544,3.805979,3.313120,4.413680,2.376270,1.599981,2.285037,3.006851


Now we find the minimum value and set it to one and set all the other values to 0

In [46]:
def nan_all_but_min(df):
    arr = df.values
    idx = np.argmin(arr, axis=1)
    newarr = np.full_like(arr, np.nan, dtype='float')
    newarr[np.arange(arr.shape[0]), idx] = arr[np.arange(arr.shape[0]), idx]
    df = pd.DataFrame(newarr, columns=df.columns, index=df.index)
    return df

In [77]:
df = equipments[['Equipment','lat','long','nombre_support']]
equipments = pd.concat([df,
                        nan_all_but_min(equipments[['Bourg l\'Evesque - La Touche - Moulin du Comte','Bréquigny','Centre','Cleunay - Arsenal - Redon',
                                                    'Francisco Ferrer - Landry - Poterie','Jeanne d\'Arc - Longs Champs - Beaulieu','Le Blosne',
                                                    'Maurepas - Bellangerais','Saint Martin','Sud gare','Thabor - Saint-Hélier - Alphonse Guérin',
                                                    'Villejean - Beauregard']]).notnull().astype('int')
                       ],axis=1)


In [78]:
equipments.head()

,Equipment,lat,long,nombre_support,Bourg l'Evesque - La Touche - Moulin du Comte,Bréquigny,Centre,Cleunay - Arsenal - Redon,Francisco Ferrer - Landry - Poterie,Jeanne d'Arc - Longs Champs - Beaulieu,Le Blosne,Maurepas - Bellangerais,Saint Martin,Sud gare,Thabor - Saint-Hélier - Alphonse Guérin,Villejean - Beauregard
0,Parking,48.112387,-1.661293,NaN,0,0,0,0,0,0,0,0,0,0,1,0
1,Parking,48.104794,-1.692262,NaN,0,0,1,0,0,0,0,0,0,0,0,0
2,Parking,48.109626,-1.672802,NaN,0,0,1,0,0,0,0,0,0,0,0,0
3,Parking,48.104185,-1.690956,NaN,0,0,1,0,0,0,0,0,0,0,0,0
4,Parking,48.106053,-1.690532,NaN,0,0,1,0,0,0,0,0,0,0,0,0


Now we do a get_dummies reversed and delete all the useless values.

In [79]:
def get_neighbourhood(row):
    for c in equipments.columns:
        if row[c]==1:
            return c
        
equipments['Neighbourhood'] = equipments.apply(get_neighbourhood, axis=1)

In [80]:
equipments = equipments[['Equipment','nombre_support','Neighbourhood']]
equipments.head()

,Equipment,nombre_support,Neighbourhood
0,Parking,NaN,Thabor - Saint-Hélier - Alphonse Guérin
1,Parking,NaN,Centre
2,Parking,NaN,Centre
3,Parking,NaN,Centre
4,Parking,NaN,Centre


Now we will count each type of equipment in each Neighbourhood

In [81]:
frames = [equipments, cultural_equipment]
equipments = pd.concat(frames)
equipments['Nombre'] = 1

In [82]:
equipments = equipments.groupby(['Equipment', 'Neighbourhood'])['nombre_support','Nombre'].sum()
equipments = equipments.fillna(0)
equipments['Number'] = equipments['nombre_support'] + equipments['Nombre']
equipments = equipments[['Number']]
equipments

Number
Equipment            Neighbourhood                                        
Cultural equipement  Bourg-L'Evêque - La Touche - Moulin du Comte     25.0
                     Bréquigny                                        16.0
                     Centre                                           18.0
                     Cleunay - Arsenal - Redon - La Courrouze         21.0
                     Francisco-Ferrer - Landry - Poterie              11.0
                     Jeanne d'Arc - Longs Champs - Beaulieu           10.0
                     Le Blosne                                        25.0
                     Maurepas - Bellangerais                          24.0
                     Saint-Martin                                      7.0
                     Sud-Gare                                         11.0
                     Thabor - Saint-Hélier -  Alphonse Guérin         15.0
                     Villejean - Beauregard                           21.0
Green Roads          Bourg l'Evesque - La Touche - Moulin du Comte   228.0
                     Bréquigny                                        74.0
                     Centre                                          451.0
                     Cleunay - Arsenal - Redon                        44.0
                     Francisco Ferrer - Landry - Poterie             242.0
                     Jeanne d'Arc - Longs Champs - Beaulieu          180.0
                     Le Blosne                                       229.0
                     Maurepas - Bellangerais                         207.0
                     Saint Martin                                    192.0
                     Sud gare                                        271.0
                     Thabor - Saint-Hélier - Alphonse Guérin         122.0
                     Villejean - Beauregard                          116.0
Parking              Centre                                          488.0
                     Saint Martin                                     33.0
                     Sud gare                                         58.0
                     Thabor - Saint-Hélier - Alphonse Guérin         167.0
bike stops           Bourg l'Evesque - La Touche - Moulin du Comte   653.0
                     Bréquigny                                       356.0
                     Centre                                         1800.0
                     Cleunay - Arsenal - Redon                        51.0
                     Francisco Ferrer - Landry - Poterie             262.0
                     Jeanne d'Arc - Longs Champs - Beaulieu          161.0
                     Le Blosne                                       403.0
                     Maurepas - Bellangerais                         214.0
                     Saint Martin                                    190.0
                     Sud gare                                        523.0
                     Thabor - Saint-Hélier - Alphonse Guérin         361.0
                     Villejean - Beauregard                          260.0
                     nombre_support                                   20.0
bus stop             Bourg l'Evesque - La Touche - Moulin du Comte    61.0
                     Bréquigny                                        34.0
                     Centre                                           87.0
                     Cleunay - Arsenal - Redon                        18.0
                     Francisco Ferrer - Landry - Poterie              65.0
                     Jeanne d'Arc - Longs Champs - Beaulieu           71.0
                     Le Blosne                                        25.0
                     Maurepas - Bellangerais                          45.0
                     Saint Martin                                     53.0
                     Sud gare                                         47.0
                     Thabor - Saint-Hélier - Alphonse Guérin          42.0
                  

# C'est pas tout a fait le format qui me convient. Vérifier avec Toronto !
# Après il faut s'occuper des données Foursquare.

# Code pour l'ancienneté des habitant et les stats de pop

In [ ]:
body = client_2551b64066e74033992250268342bdad.get_object(Bucket='courseraproject-donotdelete-pr-pstz2scfjmnlsh',Key='logement-anciennete.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

housing_seniority = pd.read_csv(body,delimiter=';')
housing_seniority = housing_seniority.loc[housing_seniority['Commune'] == 'Rennes']
housing_seniority = housing_seniority.drop(['Code INSEE', 'geolocalisation', 'Commune'], axis = 1)
housing_seniority.head()

body = client_2551b64066e74033992250268342bdad.get_object(Bucket='courseraproject-donotdelete-pr-pstz2scfjmnlsh',Key='population-par-sexe-age-et-nationalite-par-commune-2014.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

pop_stats = pd.read_csv(body,delimiter=';')
pop_stats = pop_stats.loc[pop_stats['libellé géographique'] == 'Rennes']
pop_stats = pop_stats.drop(['niveau géographique', 'code géographique', 'libellé géographique', 'CODE_DEPT', 'CODE_DEPT'], axis = 1)
pop_stats = pop_stats.drop(['NB', 'inter_codegeo1', 'EPCI', 'LIBEPCI', 'NATURE_EPCI', 'NOM_DEPT', 'CODE_REG', 'NOM_REG'], axis = 1)
pop_stats.head()